<a href="https://colab.research.google.com/github/1mranAhmed/Hug-Face-Model-Proto/blob/main/Hug_Face_Model_Proto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install git+https://github.com/huggingface/transformers -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 4.5 MB/s eta 0:00:00


In [2]:
from transformers import pipeline

Speech To Text

In [3]:
whisper = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [6]:
text = whisper('/content/Hello.mp3')

Object Detection

In [ ]:
!pip install accelerate

In [ ]:
!pip install flash_attn

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

def detect_objects_in_image(image_path_or_url):
    if image_path_or_url.startswith("http"):
        image = Image.open(requests.get(image_path_or_url, stream=True).raw)
    else:
        image = Image.open(image_path_or_url)

    processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
    model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]
        print(
                f"Detected {model.config.id2label[label.item()]} with confidence "
                f"{round(score.item(), 3)} at location {box}"
        )

image_path_or_url = input("Enter the path or URL of the image: ")
detect_objects_in_image(image_path_or_url)

Enter the path or URL of the image: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRW8BYf6K7rPa-39YxvuoDgGf5nhmjJomGAXwFcMj39EA&s
Detected backpack with confidence 0.968 at location [217.05, 105.14, 224.48, 119.42]
Detected person with confidence 1.0 at location [208.14, 96.12, 223.85, 142.93]
Detected person with confidence 0.999 at location [78.34, 66.96, 86.75, 88.73]
Detected backpack with confidence 0.968 at location [134.99, 85.76, 143.16, 97.67]
Detected car with confidence 0.986 at location [113.6, 64.17, 156.19, 79.53]
Detected car with confidence 0.999 at location [19.03, 88.62, 104.29, 138.76]
Detected person with confidence 0.999 at location [127.96, 80.99, 147.73, 116.53]
Detected person with confidence 0.995 at location [137.49, 62.7, 146.5, 83.53]
Detected car with confidence 0.951 at location [225.61, 65.37, 253.79, 89.96]


Image to Text

In [9]:
pip install diffusers --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.2 MB/s eta 0:00:00


In [2]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

img_url = 'https://media.istockphoto.com/id/473313240/photo/children-playing-in-the-park-at-playground-and-communicating.jpg?s=612x612&w=0&k=20&c=eMpRprDUvFC35nYizsBKlUPiIU9aDt8v9bKzu-ZQagI='
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

text = "a photography of"
inputs = processor(raw_image, text, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

inputs = processor(raw_image, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))


a photography of children playing on a playground structure
there are many children playing on a colorful playground set
